# SQL


In [3]:
from MySQL import MySQL_Run

MySQL_Run("show tables")

[{'Tables_in_bus_system': 'action_tour_hualien'},
 {'Tables_in_bus_system': 'admin_roles'},
 {'Tables_in_bus_system': 'admin_users'},
 {'Tables_in_bus_system': 'bus_route_stations'},
 {'Tables_in_bus_system': 'bus_routes'},
 {'Tables_in_bus_system': 'bus_routes_total'},
 {'Tables_in_bus_system': 'car_resource'},
 {'Tables_in_bus_system': 'login_state'},
 {'Tables_in_bus_system': 'reservation'},
 {'Tables_in_bus_system': 'reservation_record'},
 {'Tables_in_bus_system': 'reservation_table'},
 {'Tables_in_bus_system': 'ttcarimport'},
 {'Tables_in_bus_system': 'users'},
 {'Tables_in_bus_system': 'users_test'}]

In [4]:
import pandas as pd

Results = MySQL_Run(f"select * from reservation where user_id = '132';")
df = pd.DataFrame(Results)
df

,reservation_id,user_id,booking_time,booking_number,booking_start_station_name,booking_end_station_name,payment_method,payment_record,payment_status,review_status,dispatch_status,reservation_status,created_at,updated_at
0,24,132,2025-09-15 20:08:00,1,花蓮轉運站,花蓮港景觀橋,None,None,paid,canceled,assigned,0,2025-09-14 20:08:57,2025-09-17 22:45:46
1,26,132,2025-09-16 11:39:00,1,花蓮轉運站,將軍府,None,None,pending,canceled,not_assigned,0,2025-09-15 11:38:44,2025-09-17 22:45:46
2,27,132,2025-09-16 12:12:00,100,將軍府,四八高地,None,None,failed,canceled,not_assigned,0,2025-09-15 21:12:07,2025-09-17 22:45:46
3,28,132,2025-09-17 00:31:00,15,松園別館,花蓮觀光漁港,None,None,failed,canceled,not_assigned,0,2025-09-16 00:29:48,2025-09-17 22:45:46
4,29,132,2025-09-17 09:50:00,1,花蓮轉運站,松園別館,None,None,pending,canceled,not_assigned,0,2025-09-16 09:48:37,2025-09-17 22:45:46
5,30,132,2025-10-10 11:11:00,5,A,B,None,None,pending,canceled,not_assigned,0,2025-09-17 22:53:39,2025-09-19 16:30:48
6,31,132,2025-09-19 00:10:00,1,花蓮轉運站,花蓮港山林事業所,None,None,pending,canceled,not_assigned,0,2025-09-17 23:03:01,2025-09-19 16:34:44
7,32,132,2025-09-20 00:25:00,1,文化創意產業園區,客家文化園區,None,None,pending,canceled,not_assigned,0,2025-09-17 23:03:21,2025-09-19 16:34:47
8,33,132,2025-09-19 00:45:00,3,又一村文創園區,張家樹園,None,None,pending,pending,not_assigned,0,2025-09-18 10:05:26,2025-09-18 10:05:26
9,34,132,2025-09-27 00:15:00,1,花蓮轉運站,花蓮港山林事業所,None,None,pending,pending,not_assigned,0,2025-09-19 16:38:52,2025-09-19 16:38:52


In [ ]:
Results = MySQL_Run(f"""
UPDATE reservation
SET review_status = 'canceled'
WHERE user_id = 132;
""")
print(Results)

In [ ]:
# MySQL_Run("""DELETE FROM users
# WHERE user_id = '124';
# """)

'Query executed.'

In [ ]:
import pandas as pd

Res = "user_id, booking_time, booking_number, booking_start_station_name, booking_end_station_name"
Res2 = "user_id, booking_time, booking_number, booking_start_station_name, booking_end_station_name, review_status, payment_status"
Val = "1, '2002-06-20 10:00:00', '150', '工109', '工515'"

def Show_Resarvation(Res2):
    Results = MySQL_Run(f"select {Res2} from reservation")
    df = pd.DataFrame(Results)
    return df

def Push_Resarvation(Val):
    sql = f"insert into reservation ({Res}) values ({Val})"
    MySQL_Run(sql)
    return "Success"

# Push_Resarvation(Val)
Show_Resarvation(Res2)

In [ ]:
sql = """
SELECT user_id, booking_time, booking_number, 
        booking_start_station_name, booking_end_station_name,
        review_status, payment_status
FROM reservation where user_id = '132' order by created_at desc
"""
results = MySQL_Run(sql)
results

In [2]:

@api.post("/reservations/Canceled", tags=["Client"], summary="取消預約")
def Cancled_reservation(reservation_id: int):
    sql = f"""
    UPDATE reservation
    SET review_status = 'canceled'
    WHERE reservation_id = {reservation_id};
    """
    Results = MySQL_Run(sql)
    return {"status": "success", "sql": Results}

reservation_id = 24
sql = f"""
UPDATE reservation
SET review_status = 'canceled'
WHERE reservation_id = {reservation_id};
"""

Results = MySQL_Run(sql)
print(Results)

Query executed.


In [8]:
import redis

r = redis.Redis(host="127.0.0.1", port=6379, db=0)
r.set("foo", "bar")
print(r.get("foo"))


b'bar'


In [5]:
import pandas as pd
import pymysql

Infor = {
 'host': '43.212.245.102',
 'user': 'root',
 'port': 3306,
 'password': 'dnuiOKd217hdio',
 'database': 'Hualien_Bus',
 }

def init(Parameter):
    mydb = pymysql.connect(
        host=Parameter["host"],
        user=Parameter["user"],
        port=Parameter["port"],
        password=Parameter["password"],
        database=Parameter["database"],
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )
    return mydb

def MySQL_Run(query, Parameter=Infor):
    mydb = init(Parameter)
    mycursor = mydb.cursor()
    mycursor.execute(query)

    if query.strip().lower().startswith(("select", "show", "desc")):
        result = mycursor.fetchall()
    else:
        mydb.commit()
        result = "Query executed."

    mycursor.close()
    mydb.close()

    return (result)

MySQL_Run("""
show tables
""")

[{'Tables_in_Hualien_Bus': 'ttcarimport'}]